<a href="https://colab.research.google.com/github/Noor-Z1/Graduation-Project/blob/main/intra_subject_locomotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Pre processing


In [ ]:
# this cell is for mounting the drive to access the data

from google.colab import drive

# you can access the extracted dataset from here:
# https://drive.google.com/drive/folders/19juiVcKYgml1CUT0hJgWYWkHwghX2O5i?usp=sharing

# after having the dataset in your drive, mount google drive

drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# this cell is for preprocessing the sensors_data.txt
# to understand the columns of sensors data check: column_names.txt

import pandas as pd

# we choose 1 to 250 as the column_names.txt has defined the columns in this way
start_column = 1
end_column = 250


def preprocess_sensors_data(file_path):
    # Generate column names as a range of numbers
    column_names = list(range(start_column, end_column + 1))
    df = pd.read_csv(file_path, sep=' ')
    df.columns = column_names
    return df

def concatenate_sensor_data(base_path, ranges, drill=False):
  # Create an empty DataFrame with specified column names
    column_names = list(range(start_column, end_column + 1))
    df_concatenated = pd.DataFrame(columns=column_names)
    for range_tuple in ranges:
        for i in range_tuple[0]:
            for j in range_tuple[1]:
                file_path = f"{base_path}/S{i}-ADL{j}/S{i}-ADL{j}_sensors_data.txt"
                print(file_path)  # For demonstration, you can remove this line
                df = preprocess_sensors_data(file_path)  # Assuming preprocess_sensors_data is defined elsewhere
                df_concatenated = pd.concat([df_concatenated, df], ignore_index=True)
    if drill:
      for i in ranges[0][0]:
        # lastly include the drill run
        file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-Drill/S{}-Drill_sensors_data.txt'.format(i, i)
        print(file_path)
        df = preprocess_sensors_data(file_path)
        df_concatenated = pd.concat([df_concatenated, df])

    return df_concatenated


# ***Subject 1***

Reproducing the results of the benchmarked approach
"We train user-specific classifiers, always using as training set
the data of ADL1, ADL2, ADL3 and Drill sessions. We report classification performance for each subject on a testing set composed of
ADL4 and ADL5." https://www.sciencedirect.com/science/article/pii/S0167865512004205?ref=pdf_download&fr=RR-2&rr=86431f0ca9adc2d2

In [ ]:
# train set -> person 1 -> ADl 1, 2 and 3 and DRILL


# Define the ranges for different parts of the data
# here the first range will specify which range of subjects to look at
# and the second range will specify which range of ADLs for every subject to look at


ranges = [(range(1, 2), range(1, 4))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_train = concatenate_sensor_data(base_path, ranges, drill=True)


/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL1/S1-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL2/S1-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL3/S1-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-Drill/S1-Drill_sensors_data.txt


In [ ]:
# test set -> person 1, ADL 4 and 5

ranges = [(range(1, 2), range(4, 6))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_test = concatenate_sensor_data(base_path, ranges, drill=False)

/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL4/S1-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL5/S1-ADL5_sensors_data.txt


## *Subject 1 - Pre-Processing and EDA*

1.1 Repeating the previous value for NaN

In [ ]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,124.0,978.0,-389.0,-7.0,1014.0,199.0,124.0,968.0,123.0,...,5789.0,2908.0,1443.0,0,0,0,0,0,0,0
1,67,102.0,996.0,-440.0,-49.0,1024.0,193.0,127.0,1001.0,113.0,...,5789.0,2910.0,1440.0,0,0,0,0,0,0,0
2,100,59.0,861.0,-384.0,-9.0,1023.0,202.0,110.0,1007.0,106.0,...,5789.0,2912.0,1440.0,0,0,0,0,0,0,0
3,133,119.0,946.0,-426.0,-22.0,1026.0,188.0,98.0,1001.0,92.0,...,5791.0,2915.0,1442.0,0,0,0,0,0,0,0
4,167,99.0,972.0,-365.0,-3.0,1027.0,162.0,72.0,994.0,136.0,...,5790.0,2917.0,1445.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54960,1832015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3926.0,1613.0,869.0,0,0,0,0,0,0,0
54961,1832048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3073.0,1262.0,680.0,0,0,0,0,0,0,0
54962,1832082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2202.0,904.0,488.0,0,0,0,0,0,0,0
54963,1832115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1371.0,563.0,304.0,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_train_filled = df_train.fillna(method='ffill')

In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isnull().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(171575, 250)
35     0.295323
37     0.295323
36     0.295323
220    0.293347
221    0.293347
222    0.293347
dtype: float64


In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isna().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(171575, 250)
35     0.295323
37     0.295323
36     0.295323
220    0.293347
221    0.293347
222    0.293347
dtype: float64


even after filling with the previous value we can see the problematic columns - they never had enough 'previous' values to fill the NaN values with

In [ ]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-81.0,1057.0,10.0,-273.0,995.0,9.0,7.0,989.0,-9.0,...,5899.0,2748.0,1478.0,0,0,0,0,0,0,0
1,67,-92.0,1035.0,57.0,-294.0,985.0,0.0,26.0,998.0,-26.0,...,5900.0,2745.0,1476.0,0,0,0,0,0,0,0
2,100,-88.0,1032.0,92.0,-240.0,1014.0,32.0,7.0,995.0,-39.0,...,5902.0,2743.0,1475.0,0,0,0,0,0,0,0
3,133,-95.0,1027.0,87.0,-196.0,1002.0,38.0,38.0,1000.0,-96.0,...,5905.0,2740.0,1474.0,0,0,0,0,0,0,0
4,167,-99.0,1040.0,90.0,-225.0,992.0,30.0,71.0,1026.0,-55.0,...,5909.0,2738.0,1474.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63075,1004057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63076,1004090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63077,1004123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63078,1004157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_test_filled = df_test.fillna(method='ffill')

In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isnull().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(63080, 250)
35    0.518025
36    0.518025
37    0.518025
dtype: float64


In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isna().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(63080, 250)
35    0.518025
36    0.518025
37    0.518025
dtype: float64


ROBUST SCALER -> better with outliers
https://scikit-learn.org/0.18/auto_examples/preprocessing/plot_robust_scaling.html

a set of selected features

In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_filled[df_train_filled.columns[l]]
df_test_filtered = df_test_filled[df_test_filled.columns[l]]

In [ ]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    65643
2    43058
4    26909
5     3817
Name: count, dtype: int64
244
1    19742
4    13667
2    11664
5     2218
Name: count, dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.8111510791366906
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.80      0.87      0.83       272
           2       0.79      0.68      0.73       163
           4       0.90      0.90      0.90       106
           5       0.67      0.53      0.59        15

    accuracy                           0.81       556
   macro avg       0.79      0.75      0.76       556
weighted avg       0.81      0.81      0.81       556

Test Accuracy (KNN): 0.6808510638297872
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.57      0.86      0.69        71
           2       0.58      0.20      0.30        54
           4       0.92      0.91      0.92        54
           5       0.78      0.78      0.78         9

    accuracy                           0.68       188
   macro avg       0.71      0.69      0.67       188
weighted avg       0.68      0.68      0.65

In [ ]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.91
              precision    recall  f1-score   support

           1       0.93      0.88      0.91     19693
           2       0.83      0.90      0.86     12918
           4       1.00      0.98      0.99      8073
           5       1.00      1.00      1.00      1145

    accuracy                           0.91     41829
   macro avg       0.94      0.94      0.94     41829
weighted avg       0.91      0.91      0.91     41829



In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.93

Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.90      0.92     19742
           2       0.85      0.89      0.87     11664
           4       0.98      1.00      0.99     13667
           5       1.00      1.00      1.00      2218

    accuracy                           0.93     47291
   macro avg       0.94      0.95      0.94     47291
weighted avg       0.93      0.93      0.93     47291



Note that these results might not represent the generalization of the model so well since they are based on the same subject. Even if we do cross validation, the subset picked each time will be of the same person "that the samples produced by the same subjects are likely to be
correlated due to diverse factors. Hence, k-fold cross validation may
overestimate the performance of activity recognizers, in particular
when overlapping sliding windows are used. "
check paper: https://arxiv.org/pdf/1904.02666.pdf

1.2 Replace NaN with Spline Interpolation

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d


def interpolate_nan(df):
    # Iterate over columns
    for col in df.columns:
        # Get the column data
        col_data = df[col]

        # Find indices of NaN values
        nan_indices = col_data.isnull()

        # Perform interpolation if there are NaN values
        if nan_indices.any():
            # Find non-NaN indices
            non_nan_indices = np.arange(len(col_data))[~nan_indices]

            # Linear interpolation for NaN values at the boundaries
            interp_func = interp1d(non_nan_indices, col_data[~nan_indices], kind='linear', bounds_error=False, fill_value="extrapolate")

            # Interpolate NaN values
            x_new = np.arange(len(col_data))
            interpolated_values = interp_func(x_new)

            # Replace NaN values with interpolated values
            col_data[nan_indices] = interpolated_values[nan_indices]

            # Update the column in the DataFrame
            df[col] = col_data

    return df

# Example usage
# Replace NaN values with interpolation for df_train
df_train_interpolated = interpolate_nan(df_train)

# Replace NaN values with interpolation for df_test
df_test_interpolated = interpolate_nan(df_test)


In [ ]:
print(df_train_interpolated.shape)
nan_stats = df_train_interpolated.isna().sum().sort_values(ascending = False)/df_train_interpolated.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(171575, 250)
Series([], dtype: float64)


In [ ]:
print(df_test_interpolated.shape)
nan_stats = df_test_interpolated.isna().sum().sort_values(ascending = False)/df_test_interpolated.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(63080, 250)
Series([], dtype: float64)


In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_interpolated[df_train_interpolated.columns[l]]
df_test_filtered = df_test_interpolated[df_test_interpolated.columns[l]]

In [ ]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    65643
2    43058
4    26909
5     3817
Name: count, dtype: int64
244
1    19742
4    13667
2    11664
5     2218
Name: count, dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))

Train Accuracy (KNN): 0.8255395683453237
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.81      0.88      0.85       272
           2       0.82      0.71      0.76       163
           4       0.89      0.89      0.89       106
           5       0.71      0.67      0.69        15

    accuracy                           0.83       556
   macro avg       0.81      0.79      0.80       556
weighted avg       0.83      0.83      0.82       556

Test Accuracy (KNN): 0.6702127659574468
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.57      0.79      0.66        71
           2       0.54      0.26      0.35        54
           4       0.91      0.91      0.91        54
           5       0.70      0.78      0.74         9

    accuracy                           0.67       188
   macro avg       0.68      0.68      0.66       188
weighted avg       0.66      0.67      0.65

In [ ]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.92
              precision    recall  f1-score   support

           1       0.94      0.89      0.91     19693
           2       0.83      0.91      0.87     12918
           4       1.00      0.98      0.99      8073
           5       1.00      1.00      1.00      1145

    accuracy                           0.92     41829
   macro avg       0.94      0.94      0.94     41829
weighted avg       0.92      0.92      0.92     41829



In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.93

Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.91      0.92     19742
           2       0.87      0.88      0.87     11664
           4       0.98      1.00      0.99     13667
           5       1.00      1.00      1.00      2218

    accuracy                           0.93     47291
   macro avg       0.94      0.95      0.94     47291
weighted avg       0.93      0.93      0.93     47291



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# *Subject 2*

In [ ]:
# train set -> person 2 -> ADl 1, 2 and 3 and DRILL


# Define the ranges for different parts of the data
# here the first range will specify which range of subjects to look at
# and the second range will specify which range of ADLs for every subject to look at


ranges = [(range(2, 3), range(1, 4))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_train = concatenate_sensor_data(base_path, ranges, drill=True)


/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL1/S2-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL2/S2-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL3/S2-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-Drill/S2-Drill_sensors_data.txt


In [ ]:
# test set -> person 2, ADL 4 and 5

ranges = [(range(2, 3), range(4, 6))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_test = concatenate_sensor_data(base_path, ranges, drill=False)

/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL4/S2-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL5/S2-ADL5_sensors_data.txt


2.1 Repeating the previous value for NaN

In [ ]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-62.0,1052.0,-6.0,76.0,968.0,374.0,-484.0,392.0,783.0,...,4582.0,1913.0,1487.0,0,0,0,0,0,0,0
1,67,-31.0,1053.0,-27.0,90.0,932.0,385.0,-515.0,393.0,778.0,...,4578.0,1914.0,1486.0,0,0,0,0,0,0,0
2,100,-55.0,1029.0,-30.0,69.0,971.0,360.0,-533.0,387.0,755.0,...,4575.0,1915.0,1485.0,0,0,0,0,0,0,0
3,133,-64.0,1043.0,-28.0,102.0,952.0,384.0,-522.0,389.0,770.0,...,4572.0,1917.0,1484.0,0,0,0,0,0,0,0
4,167,-43.0,1040.0,-37.0,92.0,946.0,357.0,-558.0,398.0,747.0,...,4570.0,1920.0,1483.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53392,1779749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3300.0,1213.0,1121.0,0,0,0,0,0,0,0
53393,1779782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2511.0,924.0,853.0,0,0,0,0,0,0,0
53394,1779816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1725.0,635.0,586.0,0,0,0,0,0,0,0
53395,1779849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,993.0,366.0,337.0,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_train_filled = df_train.fillna(method='ffill')

In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isnull().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(160605, 250)
35     0.224464
37     0.224464
36     0.224464
153    0.067103
152    0.067103
151    0.067103
150    0.067103
154    0.067103
dtype: float64


In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isna().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(160605, 250)
35     0.224464
37     0.224464
36     0.224464
153    0.067103
152    0.067103
151    0.067103
150    0.067103
154    0.067103
dtype: float64


even after filling with the previous value we can see the problematic columns - they never had enough 'previous' values to fill the NaN values with

In [ ]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-59.0,1050.0,-17.0,93.0,996.0,-98.0,16.0,952.0,394.0,...,5043,1972,1690,0,0,0,0,0,0,0
1,67,-93.0,1028.0,-71.0,63.0,965.0,-155.0,13.0,933.0,382.0,...,5043,1971,1693,0,0,0,0,0,0,0
2,100,-103.0,1014.0,-123.0,68.0,966.0,-151.0,40.0,915.0,327.0,...,5044,1971,1696,0,0,0,0,0,0,0
3,133,-76.0,1006.0,-111.0,79.0,978.0,-172.0,70.0,930.0,268.0,...,5045,1971,1699,0,0,0,0,0,0,0
4,167,-46.0,1031.0,-36.0,77.0,1027.0,-197.0,60.0,948.0,307.0,...,5047,1971,1701,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64567,1060689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3226,1366,883,0,0,0,0,0,0,0
64568,1060723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2426,1027,663,0,0,0,0,0,0,0
64569,1060756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1637,693,447,0,0,0,0,0,0,0
64570,1060789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,910,385,249,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_test_filled = df_test.fillna(method='ffill')

In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isnull().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(64572, 250)
Series([], dtype: float64)


In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isna().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(64572, 250)
Series([], dtype: float64)


In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_filled[df_train_filled.columns[l]]
df_test_filtered = df_test_filled[df_test_filled.columns[l]]

In [ ]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    61657
2    38408
4    28878
5     3016
Name: count, dtype: int64
244
1    17844
4    15047
2    13697
5     1898
Name: count, dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.8174904942965779
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.80      0.89      0.85       248
           2       0.76      0.68      0.72       151
           4       0.94      0.87      0.90       114
           5       0.64      0.54      0.58        13

    accuracy                           0.82       526
   macro avg       0.79      0.75      0.76       526
weighted avg       0.82      0.82      0.82       526

Test Accuracy (KNN): 0.71875
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.70      0.78      0.74        79
           2       0.50      0.34      0.41        47
           4       0.89      0.95      0.92        60
           5       0.43      0.50      0.46         6

    accuracy                           0.72       192
   macro avg       0.63      0.64      0.63       192
weighted avg       0.70      0.72      0.70       192


In [ ]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.92
              precision    recall  f1-score   support

           1       0.93      0.90      0.91     18497
           2       0.84      0.91      0.87     11523
           4       1.00      0.97      0.98      8663
           5       0.92      0.98      0.95       905

    accuracy                           0.92     39588
   macro avg       0.92      0.94      0.93     39588
weighted avg       0.92      0.92      0.92     39588



In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.91

Classification Report:
              precision    recall  f1-score   support

           1       0.91      0.86      0.89     17844
           2       0.83      0.90      0.86     13697
           4       0.99      0.99      0.99     15047
           5       0.98      0.96      0.97      1898

    accuracy                           0.91     48486
   macro avg       0.93      0.93      0.93     48486
weighted avg       0.92      0.91      0.91     48486



# *Subject 3*

In [43]:
# train set -> person 3 -> ADl 1, 2 and 3 and DRILL


# Define the ranges for different parts of the data
# here the first range will specify which range of subjects to look at
# and the second range will specify which range of ADLs for every subject to look at


ranges = [(range(3, 4), range(1, 4))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_train = concatenate_sensor_data(base_path, ranges, drill=True)


/content/gdrive/MyDrive/Opportunity_extracted/data/S3-ADL1/S3-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S3-ADL2/S3-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S3-ADL3/S3-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S3-Drill/S3-Drill_sensors_data.txt


In [44]:
# test set -> person 2, ADL 4 and 5

ranges = [(range(3, 4), range(4, 6))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_test = concatenate_sensor_data(base_path, ranges, drill=False)

/content/gdrive/MyDrive/Opportunity_extracted/data/S3-ADL4/S3-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S3-ADL5/S3-ADL5_sensors_data.txt


3.1 Repeating the previous value for NaN

In [45]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-9.0,-40.0,1011.0,-58.0,1001.0,140.0,-345.0,78.0,942.0,...,4805.0,1531.0,978.0,0,0,0,0,0,0,0
1,67,0.0,-25.0,1016.0,-82.0,1012.0,136.0,-338.0,74.0,989.0,...,4804.0,1529.0,979.0,0,0,0,0,0,0,0
2,100,-14.0,-24.0,1018.0,-96.0,1005.0,131.0,-347.0,84.0,962.0,...,4804.0,1528.0,979.0,0,0,0,0,0,0,0
3,133,-16.0,-23.0,1046.0,-91.0,1013.0,133.0,-330.0,77.0,965.0,...,4804.0,1528.0,980.0,0,0,0,0,0,0,0
4,167,-12.0,-14.0,1014.0,-94.0,998.0,123.0,-351.0,84.0,944.0,...,4806.0,1531.0,982.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70922,2364076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3865.0,740.0,987.0,0,0,0,0,0,0,0
70923,2364110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2979.0,570.0,760.0,0,0,0,0,0,0,0
70924,2364143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2088.0,400.0,533.0,0,0,0,0,0,0,0
70925,2364176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1249.0,239.0,319.0,0,0,0,0,0,0,0


In [46]:
# Fill NaN values with the previous non-NaN value
df_train_filled = df_train.fillna(method='ffill')

In [47]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isnull().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(162689, 250)
13    0.037188
12    0.037188
11    0.037188
dtype: float64


In [49]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isna().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(162689, 250)
13    0.037188
12    0.037188
11    0.037188
dtype: float64


even after filling with the previous value we can see the problematic columns - they never had enough 'previous' values to fill the NaN values with

In [50]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-101.0,1001.0,269.0,-275.0,976.0,223.0,-532.0,94.0,833.0,...,5061.0,1597.0,1421.0,0,0,0,0,0,0,0
1,67,-121.0,1021.0,277.0,-294.0,965.0,202.0,-534.0,73.0,852.0,...,5063.0,1596.0,1421.0,0,0,0,0,0,0,0
2,100,-150.0,1019.0,277.0,-297.0,955.0,201.0,-530.0,81.0,869.0,...,5064.0,1595.0,1422.0,0,0,0,0,0,0,0
3,133,-129.0,999.0,278.0,-312.0,956.0,190.0,-522.0,76.0,843.0,...,5066.0,1595.0,1424.0,0,0,0,0,0,0,0
4,167,-132.0,991.0,288.0,-291.0,970.0,196.0,-523.0,77.0,858.0,...,5066.0,1596.0,1426.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54169,882991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
54170,883025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
54171,883058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
54172,883091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [51]:
# Fill NaN values with the previous non-NaN value
df_test_filled = df_test.fillna(method='ffill')

In [52]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isnull().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(54174, 250)
Series([], dtype: float64)


In [53]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isna().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(54174, 250)
Series([], dtype: float64)


In [57]:
l = [4,5,6, 7,8,9, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_filled[df_train_filled.columns[l]]
df_test_filtered = df_test_filled[df_test_filled.columns[l]]

In [58]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    84709
2    37376
4    19429
5     5477
Name: count, dtype: int64
244
1    19440
2    13502
4     9173
5     3659
Name: count, dtype: int64


In [59]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.8395904436860068
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.81      0.95      0.87       334
           2       0.87      0.59      0.70       152
           4       0.93      0.87      0.90        79
           5       0.86      0.86      0.86        21

    accuracy                           0.84       586
   macro avg       0.87      0.82      0.83       586
weighted avg       0.85      0.84      0.83       586

Test Accuracy (KNN): 0.7527472527472527
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.70      0.83      0.76        83
           2       0.70      0.46      0.55        50
           4       0.86      0.89      0.87        35
           5       1.00      1.00      1.00        14

    accuracy                           0.75       182
   macro avg       0.81      0.79      0.80       182
weighted avg       0.75      0.75      0.74

In [60]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.91
              precision    recall  f1-score   support

           1       0.95      0.90      0.92     25413
           2       0.79      0.89      0.84     11213
           4       0.99      0.97      0.98      5829
           5       1.00      1.00      1.00      1643

    accuracy                           0.91     44098
   macro avg       0.93      0.94      0.94     44098
weighted avg       0.92      0.91      0.91     44098



In [62]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.89

Classification Report:
              precision    recall  f1-score   support

           1       0.88      0.87      0.87     19440
           2       0.81      0.87      0.84     13502
           4       0.97      0.92      0.95      9173
           5       1.00      0.97      0.98      3659

    accuracy                           0.89     45774
   macro avg       0.92      0.91      0.91     45774
weighted avg       0.89      0.89      0.89     45774



# *Subject 4*

In [63]:
# train set -> person 4 -> ADl 1, 2 and 3 and DRILL


# Define the ranges for different parts of the data
# here the first range will specify which range of subjects to look at
# and the second range will specify which range of ADLs for every subject to look at


ranges = [(range(4, 5), range(1, 4))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_train = concatenate_sensor_data(base_path, ranges, drill=True)


/content/gdrive/MyDrive/Opportunity_extracted/data/S4-ADL1/S4-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S4-ADL2/S4-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S4-ADL3/S4-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S4-Drill/S4-Drill_sensors_data.txt


In [64]:
# test set -> person 2, ADL 4 and 5

ranges = [(range(4, 5), range(4, 6))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_test = concatenate_sensor_data(base_path, ranges, drill=False)

/content/gdrive/MyDrive/Opportunity_extracted/data/S4-ADL4/S4-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S4-ADL5/S4-ADL5_sensors_data.txt


4.1 Repeating the previous value for NaN

In [65]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-18.0,1047.0,69.0,-9.0,931.0,462.0,450.0,861.0,367.0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,67,-12.0,1043.0,52.0,-38.0,924.0,463.0,502.0,839.0,367.0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
2,100,0.0,1021.0,48.0,-38.0,929.0,443.0,583.0,873.0,361.0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,133,-10.0,1031.0,59.0,-33.0,918.0,464.0,664.0,833.0,389.0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,167,-19.0,1050.0,59.0,-10.0,919.0,466.0,642.0,809.0,504.0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45454,1515152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
45455,1515185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
45456,1515218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
45457,1515252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [66]:
# Fill NaN values with the previous non-NaN value
df_train_filled = df_train.fillna(method='ffill')

In [67]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isnull().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(139913, 250)
Series([], dtype: float64)


In [69]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isna().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.01])

(139913, 250)
Series([], dtype: float64)


even after filling with the previous value we can see the problematic columns - they never had enough 'previous' values to fill the NaN values with

In [70]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,147.0,1015.0,-60.0,45.0,956.0,387.0,322.0,49.0,867.0,...,5763.0,2787.0,1129.0,0,0,0,0,0,0,0
1,67,144.0,1035.0,-72.0,65.0,961.0,376.0,325.0,55.0,883.0,...,5755.0,2755.0,1127.0,0,0,0,0,0,0,0
2,100,148.0,1035.0,-63.0,63.0,970.0,383.0,339.0,45.0,921.0,...,5746.0,2715.0,1127.0,0,0,0,0,0,0,0
3,133,144.0,1028.0,-43.0,32.0,938.0,377.0,336.0,44.0,941.0,...,5737.0,2670.0,1128.0,0,0,0,0,0,0,0
4,167,141.0,1040.0,-47.0,20.0,964.0,364.0,338.0,35.0,953.0,...,5729.0,2622.0,1130.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52750,1017390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
52751,1017423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
52752,1017456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
52753,1017490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [77]:
# Fill NaN values with the previous non-NaN value
df_test_filled = df_test.fillna(method='ffill')
df_test_filled = df_test_filled.fillna(method='bfill')

In [78]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isnull().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(52755, 250)
Series([], dtype: float64)


In [79]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isna().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(52755, 250)
Series([], dtype: float64)


In [80]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_filled[df_train_filled.columns[l]]
df_test_filtered = df_test_filled[df_test_filled.columns[l]]

In [81]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    59057
2    27494
4    17910
5     3591
Name: count, dtype: int64
244
1    17305
2    14014
4     5839
5     1719
Name: count, dtype: int64


In [82]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.814385150812065
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.79      0.93      0.86       239
           2       0.76      0.50      0.61       103
           4       0.94      0.87      0.90        75
           5       0.85      0.79      0.81        14

    accuracy                           0.81       431
   macro avg       0.84      0.77      0.80       431
weighted avg       0.81      0.81      0.80       431

Test Accuracy (KNN): 0.5454545454545454
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.49      0.77      0.60        64
           2       0.50      0.21      0.30        61
           4       0.81      0.91      0.86        23
           5       0.33      0.17      0.22         6

    accuracy                           0.55       154
   macro avg       0.53      0.51      0.49       154
weighted avg       0.54      0.55      0.50 

In [83]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.91
              precision    recall  f1-score   support

           1       0.95      0.89      0.92     17718
           2       0.79      0.91      0.84      8248
           4       0.99      0.98      0.99      5373
           5       1.00      0.99      1.00      1077

    accuracy                           0.91     32416
   macro avg       0.93      0.94      0.94     32416
weighted avg       0.92      0.91      0.91     32416



In [85]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

           1       0.85      0.87      0.86     17305
           2       0.83      0.83      0.83     14014
           4       0.98      0.96      0.97      5839
           5       0.99      0.96      0.97      1719

    accuracy                           0.87     38877
   macro avg       0.91      0.90      0.91     38877
weighted avg       0.87      0.87      0.87     38877

